# Inserts


### The `insert()` construct

- the `insert()` constructs directly corresponds to a SQL `INSERT` statement.

In [63]:
from datetime import datetime
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Mapped, mapped_column, DeclarativeBase, MappedAsDataclass


class Base(MappedAsDataclass, DeclarativeBase):
    pass


class User(Base):  # class can be called whatever we want, no impact on the table name
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True, init=False)
    name: Mapped[str]
    full_name: Mapped[str | None]
    created_at: Mapped[datetime] = mapped_column(init=False, server_default=func.now())


engine = create_engine("sqlite://", echo=True)
with engine.begin() as conn:
    Base.metadata.create_all(conn)

2024-12-08 22:01:44,959 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-08 22:01:44,960 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-12-08 22:01:44,960 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-08 22:01:44,961 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-12-08 22:01:44,962 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-08 22:01:44,963 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	full_name VARCHAR, 
	created_at DATETIME DEFAULT (CURRENT_TIMESTAMP) NOT NULL, 
	PRIMARY KEY (id)
)


2024-12-08 22:01:44,963 INFO sqlalchemy.engine.Engine [no key 0.00049s] ()
2024-12-08 22:01:44,964 INFO sqlalchemy.engine.Engine COMMIT


**About `func`**

- `func` is a namespace that provides a way to generate SQL function expressions. It allows you to call SQL functions (both built-in and custom functions) within SQLAlchemy queries
- Examples: `COUNT`, `AVG`, `MAX`, `MIN`, `SUM`.
- You can use func expressions as part of SQLAlchemy query filters, columns, or other clauses.


- the `insert()` function starts with a single argument: the table, or table-representing ORM entity.
- in this case, it's the `User` class

In [64]:
from sqlalchemy import insert

insert_stmt = insert(User)

print(insert_stmt)  # get this generic insert with columns

INSERT INTO user_account (id, name, full_name, created_at) VALUES (:id, :name, :full_name, :created_at)


- next we further indicate a VALUES clause to be included in the `INSERT` statement, usin the `insert.values()` method, which accepts keys that match database column names

In [65]:
insert_stmt = insert_stmt.values(name="Walt", full_name="Walther White")

with engine.begin() as conn:
    conn.execute(insert_stmt)

2024-12-08 22:01:44,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-08 22:01:44,980 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, full_name) VALUES (?, ?)
2024-12-08 22:01:44,980 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ('Walt', 'Walther White')
2024-12-08 22:01:44,981 INFO sqlalchemy.engine.Engine COMMIT


_Note that the `?` are placeholders used to represent values that will be bound to the query at **execution time**. They can be displayed differtly according to the chosen database.

### INSERT using `execute()`

- `.values()` is used to produce a specific VALUES clause.
- For simple cases, we usually rely opon SQLAlchemy itself to generate this for us.
- For this, we can pass a `dict` or `list[dict]` to the `.execute()` method to accompany the statement.
- SQLAlchemy will then INSERT lists of dictionnaties **in bulk**.
- The convention of "keys match column names" is the same as that if the `insert.values()`

In [66]:
with engine.begin() as conn:
    conn.execute(
        insert(User),
        [
            {"name": "Sandy", "full_name": "Sandy Kilo"},
            {"name": "Gary", "full_name": "Gary Gareau"},
        ],
    )

2024-12-08 22:01:44,987 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-08 22:01:44,988 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, full_name) VALUES (?, ?)
2024-12-08 22:01:44,989 INFO sqlalchemy.engine.Engine [generated in 0.00039s] [('Sandy', 'Sandy Kilo'), ('Gary', 'Gary Gareau')]
2024-12-08 22:01:44,990 INFO sqlalchemy.engine.Engine COMMIT
